In [2]:
import numpy as np
from aocd.models import Puzzle

# Tips

## Define the object
- `puzzle = Puzzle(year=2017, day=20)` 
- `Puzzle(2017, 20) at 0x107322978 - Particle Swarm>`
- get the input `puzzle.input_data`
- submit by setting:
  - `puzzle.answer_a = value_a`
  - `puzzle.answer_b = value_b`

## Transform to list variables on multiple lines: 
 - t = '''asd
        asd 
        asd 
        asd'''.split('\n')

## Map a string list to integer 
- `map(int, list)`
- `np.array(p.input_data.split('\n'), dtype='int')`

## Day 1

In [2]:
puzzle = Puzzle(year=2024, day=1)


def get_distance(a: np.array, b: np.array) -> int:
    a.sort()
    b.sort()
    return np.sum(np.abs(a - b))


def get_similarity(a: np.array, b: np.array) -> int:
    sim = 0
    for el in a:
        sim += el * np.sum(b == el)
    return sim


# tests
data = puzzle.examples[0].input_data.split("\n")
data = np.array([list(map(int, line.split())) for line in data])
assert get_distance(data[:, 0], data[:, 1]) == 11
assert get_similarity(data[:, 0], data[:, 1]) == 31

# part a
data = puzzle.input_data.split("\n")
data = np.array([list(map(int, line.split())) for line in data])
puzzle.answer_a = get_distance(data[:, 0], data[:, 1])

# part b
puzzle.answer_b = get_similarity(data[:, 0], data[:, 1])

coerced int64 value np.int64(1580061) for 2024/01 to '1580061'
coerced int64 value np.int64(23046913) for 2024/01 to '23046913'


# Day 2

In [3]:
puzzle = Puzzle(year=2024, day=2)

test_data = [
    list(map(int, l.split())) for l in puzzle.examples[0].input_data.split("\n")
]
data = [list(map(int, l.split())) for l in puzzle.input_data.split("\n")]

In [ ]:
def possible_bad_level(inc: np.array, dec: np.array, gaps: np.array):
    """Find the indices that could be removed to make the list safe.

    Args:
        inc (np.array): boolean where the list is increasing
        dec (np.array): boolean where the list is decreasing
        gaps (np.array): boolean where the gap are within the [0, 3] limits

    Returns:
        list: Possible bad levels to try
    """

    inc_ix = np.where(~inc)[0]
    dec_ix = np.where(~dec)[0]
    gaps_ix = np.where(~gaps)[0]

    # we add the index where it occurs and the next
    # 9 8 7 4 5 -> increase at index 3 and must remove index 3
    # 9 8 7 6 7 -> increase at index 3 but must remove 4
    # same ideas is valid for the other conditions
    bad_level = set()

    # can't fix more than 2 issues for a condition
    # there is for sure a way to optimize this better
    if len(inc_ix) == 1:
        bad_level.update([inc_ix[0], inc_ix[0] + 1])

    if len(dec_ix) == 1:
        bad_level.update([dec_ix[0], dec_ix[0] + 1])

    if len(gaps_ix) and len(gaps_ix) <= 2:
        bad_level.update([gaps_ix[0], gaps_ix[0] + 1])

    return bad_level


def safe_list(data: np.array, part_b=False) -> bool:
    """Check if the list is safe based on monotonicity and gap jump.

    Args:
        data (np.array): input list to validate
        part_b (bool, optional): Also attempt to validate by removing a single element.
            Defaults to False.

    Returns:
        bool: True if the list is safe otherwise False
    """
    diff = np.diff(data)
    inc = diff > 0
    dec = diff < 0
    safe_gap = (np.abs(diff) >= 0) & (np.abs(diff) <= 3)

    if not (all(inc) or all(dec)) or not all(safe_gap):
        if part_b and (indices := possible_bad_level(inc, dec, safe_gap)):
            return any(safe_list(np.delete(data, ix)) for ix in indices)
        return False

    return True


# test cases
assert sum([safe_list(d) for d in test_data]) == 2
assert sum([safe_list(d, part_b=True) for d in test_data]) == 4

puzzle.answer_a = sum([safe_list(d) for d in data])
puzzle.answer_b = sum([safe_list(d, part_b=True) for d in data])

# Day 3

In [76]:
puzzle = Puzzle(year=2024, day=3)

example_data = puzzle.examples[0].input_data
example_data_b = (
    "xmul(2,4)&mul[3,7]!^don't()_mul(5,5)+mul(32,64](mul(11,8)undo()?mul(8,5))"
)
data = puzzle.input_data

In [77]:
def eval_mul(data, part_b=False):
    mul = 0
    process = True

    for i in range(0, len(data)):
        next_op = data[i : i + 4]
        possible_stop = data[i : i + 7]

        if part_b and possible_stop == "don't()":
            process = False
        elif next_op == "do()":
            process = True
        elif process and next_op == "mul(":
            l = data[i + 4 :].partition(",")[0]
            r = data[i + 4 + len(l) + 1 :].partition(")")[0]

            if l.isdigit() and r.isdigit() and " " not in l and " " not in r:
                mul += int(l) * int(r)
    return mul


# tests
assert eval_mul(example_data) == 161
assert eval_mul(example_data_b, part_b=True) == 48

puzzle.answer_a = eval_mul(data)
puzzle.answer_b = eval_mul(data, part_b=True)

# Day 4

In [ ]:
puzzle = Puzzle(year=2024, day=4)

example_data = puzzle.examples[0].input_data
data = puzzle.input_data

# Day 5

# Day 6

# Day 7

# Day 8

# Day 9

# Day 10

# Day 11

# Day 12

# Day 13

# Day 14

# Day 15

# Day 16

# Day 17

# Day 18

# Day 19

# Day 20

# Day 21

# Day 22

# Day 23

# Day 24

# Day 25